# Homework 06: Support Vector Machine Classification
## Kerem Girenes
### April 28, 2022

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.spatial.distance as dt
import cvxopt as cvx

ModuleNotFoundError: No module named 'cvxopt'

## Importing Data

In [1]:
images = np.genfromtxt("hw06_data_set_images.csv", delimiter=",")
labels = np.genfromtxt("hw06_data_set_labels.csv", delimiter=",")

NameError: name 'np' is not defined

## Dividing the Data Set

In [4]:
X_train = images[:1000]
X_test = images[1000:]
y_train = labels[:1000].astype(int)
y_test = labels[1000:].astype(int)

# two distinct classes
K = 2

# N_train = N_test
N = X_train.shape[0]

# Pixel Count
PX = X_train.shape[1]

# Number of Bins
NO_BINS = 64

## Construct Color Histograms for Training and Test Data

In [5]:
bins = np.linspace(0, 1, NO_BINS)

H_train = np.zeros((N, NO_BINS))
H_test = np.zeros((N, NO_BINS))

for xi in range(N):
    for pi in range(PX):
        # train histogram
        b = (X_train[xi][pi]/4).astype(int)
        H_train[xi][b] += 1
        # test histogram
        b = (X_test[xi][pi]/4).astype(int)
        H_test[xi][b] += 1
    # normalize histograms
    H_train[xi] = H_train[xi] / PX
    H_test[xi] = H_test[xi] / PX
    
print(H_train[0:5, 0:5])
print(H_test[0:5, 0:5])

[[0.86479592 0.00127551 0.         0.00255102 0.        ]
 [0.66836735 0.         0.00127551 0.00127551 0.        ]
 [0.66454082 0.00637755 0.00382653 0.00765306 0.00892857]
 [0.65816327 0.00765306 0.00892857 0.00127551 0.00382653]
 [0.5625     0.00255102 0.00255102 0.00127551 0.        ]]
[[0.68239796 0.00255102 0.00127551 0.00127551 0.00127551]
 [0.69770408 0.01658163 0.00510204 0.00382653 0.01020408]
 [0.73341837 0.02678571 0.01530612 0.00510204 0.00637755]
 [0.63903061 0.00892857 0.00255102 0.00127551 0.        ]
 [0.75382653 0.00765306 0.00127551 0.00127551 0.        ]]


## Construct Kernel Matrices for Training and Test Data

In [6]:
def h_kernel(hi, hj):
    res = 0
    for l in range(NO_BINS):
        res += min(hi[l], hj[l])
    return res

K_train = np.zeros((N, N))
K_test = np.zeros((N, N))

for i in range(N):
    for j in range(N):
        K_train[i][j] = h_kernel(H_train[i], H_train[j])
        K_test[i][j] = h_kernel(H_test[i], H_train[j])
        
print(K_train[0:5, 0:5])
print(K_test[0:5, 0:5])

[[1.         0.72321429 0.77040816 0.75382653 0.62755102]
 [0.72321429 1.         0.73086735 0.78571429 0.68622449]
 [0.77040816 0.73086735 1.         0.84056122 0.70153061]
 [0.75382653 0.78571429 0.84056122 1.         0.76403061]
 [0.62755102 0.68622449 0.70153061 0.76403061 1.        ]]
[[0.77806122 0.80867347 0.82142857 0.88647959 0.79209184]
 [0.79464286 0.76403061 0.84566327 0.86607143 0.77933673]
 [0.8380102  0.74362245 0.85714286 0.83035714 0.68877551]
 [0.71556122 0.84438776 0.75       0.83418367 0.75765306]
 [0.84438776 0.76785714 0.82397959 0.84183673 0.73469388]]


## Train a SVM Classifier and Calculate the Confusion Matrices

In [ ]:
def train_svm(X_set, y_set, N, s, C, epsilon):
    yyK = np.matmul(y_set[:,None], y_set[None,:]) * K_train

    P = cvx.matrix(yyK)
    q = cvx.matrix(-np.ones((N, 1)))
    G = cvx.matrix(np.vstack((-np.eye(N), np.eye(N))))
    h = cvx.matrix(np.vstack((np.zeros((N, 1)), C * np.ones((N, 1)))))
    A = cvx.matrix(1.0 * y_set[None,:])
    b = cvx.matrix(0.0)

    # use cvxopt library to solve QP problems
    result = cvx.solvers.qp(P, q, G, h, A, b)
    alpha = np.reshape(result["x"], N)
    alpha[alpha < C * epsilon] = 0
    alpha[alpha > C * (1 - epsilon)] = C

    # find bias parameter
    support_indices, = np.where(alpha != 0)
    active_indices, = np.where(np.logical_and(alpha != 0, alpha < C))
    w0 = np.mean(y_set[active_indices] * (1 - np.matmul(yyK[np.ix_(active_indices, support_indices)], alpha[support_indices])))
    
    return alpha, w0

def pred(K_train, K_test, s, C, epsilon):
    f_pred_train = []
    f_pred_test = []
    alpha, w0 = train_svm(X_train, y_train, N, s, C, epsilon)
    # training predictions
    f_pred_train = np.matmul(K_train, y_train[:, None] * alpha[:, None]) + w0
    # test predictions
    f_pred_test = np.matmul(K_test, y_train[:, None] * alpha[:, None]) + w0
    # prediction labels
    y_pred_train = 2 * (f_pred_train > 0.0) - 1
    y_pred_test = 2 * (f_pred_test > 0.0) - 1
    
    return y_pred_train, y_pred_test
    

C = 10
s = 1
epsilon = 0.001

y_pred_train, y_pred_test = pred(K_train, K_test, s, C, epsilon)

print("Training Confusion Matrix:")
print(pd.crosstab(np.reshape(y_pred_train, N), y_train, rownames = ["y_predicted"], colnames = ["y_train"]))
print("Test Confusion Matrix:")
print(pd.crosstab(np.reshape(y_pred_test, N), y_test, rownames = ["y_predicted"], colnames = ["y_test"]))

## Train SVM by Varying C, Plot Accuracies

In [ ]:
C = [10**-1, 10**-.5, 10**0, 10**.5, 10**1, 10**1.5, 10**2, 10**2.5, 10**3]

training_scores = []
test_scores = []

def accuracy(y_pred, y_truth):
    score = 0.0
    for i in range(len(y_truth)):
        score += (y_pred[i] == y_truth[i]) * 1.0
    return score / len(y_truth)

for c in C:
    y_pred_train, y_pred_test = pred(K_train, K_test, s, c, epsilon)
    training_scores.append(accuracy(y_pred_train, y_train))
    test_scores.append(accuracy(y_pred_test, y_test))
    
x_axis_vals = ["10^-1", "10^-0.5", "1", "10^0.5", "10^1", "10^1.5", "10^2", "10^2.5", "10^3"]
plt.figure(figsize=(20,6))
plt.plot(x_axis_vals, training_scores, "-ob", markersize=4, label='training')
plt.plot(x_axis_vals, test_scores, "-or", markersize=4, label='test')
plt.xlabel("Regularization parameter (C)")
plt.ylabel("Accuracy")
plt.legend(loc='upper left')
plt.show()